In [2]:
import numpy as np
import pandas as pd
import os
import tqdm
import sqlalchemy as sqlalchemy
from sqlalchemy import create_engine
import sqlite3
import time

In [3]:
def Get_File_List(Check_year):
    File_List = []
    Data_Location = "C:\\Users\\20192830\\Documents\\Data Challenge 2\\Data-Challenge-2\\Data_Challenge_2_Data\\" #Enter your data location to the files
    Main_Folder = os.listdir(Data_Location)
    for month_folder in Main_Folder:
        year = month_folder.split('-')[0]
        month = month_folder.split('-')[1]
        Sub_Folder = Data_Location + month_folder
        if(int(year) == int(Check_year)): #set ==, > or < to select the years
            for district_folder in os.listdir(Sub_Folder):
                if district_folder.endswith('-street.csv'):
                    filename = Sub_Folder + "\\" + district_folder
                    File_List.append(filename)
    return File_List

In [3]:
#CREATE DATABASE (RUN GETFILELIST FIRST)
col_names = ["Crime ID", "Month", "Reported by", "Falls within", "Longitude", "Latitude", "Location", "LSOA code",
             "LSOA name", "Crime type", "Last outcome category", "Context"]
Crime_df = pd.DataFrame()
Crime_df = Crime_df.reindex(Crime_df.columns.union(col_names), axis=1)

Data = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

conn = sqlite3.connect(r'C:\\Users\\20192830\\Documents\\Data Challenge 2\\DC22_DB.db') #Specify output 
#takes +- 15 minutes
for year in tqdm.tqdm(Data):
    File_List = Get_File_List(year)
    for File in File_List:
        Crime_df = pd.read_csv(File)
        Crime_df = Crime_df.dropna(subset=['Longitude', 'Latitude'])
        Crime_df[['Year','Month']] = Crime_df['Month'].str.split('-',expand=True)  
        Crime_df = Crime_df[["Crime ID", "Year", "Month", "Reported by", "Falls within", "Longitude", "Latitude", "Location", "LSOA code",
                             "LSOA name", "Crime type", "Last outcome category", "Context"]]
        Crime_df.to_sql('Street', conn, if_exists='append', index=False)


  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\20192830\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
100%|██████████| 10/10 [10:24<00:00, 62.40s/it]


In [5]:
def get_amount_data():
    con = sqlite3.connect('DC2_DB.db')
    query = f'''SELECT Year, Month, `Falls within` AS "Region", COUNT(*) AS "Crime_Number"
                FROM Street_fixed
                GROUP BY Year, Month, `Falls within`
             '''
    sql_query = pd.read_sql_query(query, con)
    df = pd.DataFrame(sql_query)
    
    df['Month'] = df.Month.astype(str).str.pad(2,fillchar='0')
    df['Year'] = df['Year'].astype(str)

    df['Date'] = pd.to_datetime(df['Year'] + df['Month'], format='%Y%m')
    del df['Year']
    del df ['Month']
    return df

Average_Crimes = get_amount_data()
Average_Crimes_Train = Average_Crimes.loc[Average_Crimes["Date"] < pd.to_datetime(2019, format='%Y')]
Average_Crimes_Test = Average_Crimes.loc[(Average_Crimes["Date"] >= pd.to_datetime(2019, format='%Y')) & (Average_Crimes["Date"] < pd.to_datetime(2020, format='%Y'))]


In [8]:

#add sub tables
con = sqlite3.connect('DC22_DB.db')
query = f'''SELECT DISTINCT(`Falls within`)
                FROM Street_fixed
             '''
sql_query = pd.read_sql_query(query, con)
df = pd.DataFrame(sql_query)
df.to_sql('Region', con, if_exists='replace', index=False)

#add sub tables
con = sqlite3.connect('DC22_DB.db')
query = f'''SELECT DISTINCT(`Crime type`)
                FROM Street_fixed
             '''
sql_query = pd.read_sql_query(query, con)
df = pd.DataFrame(sql_query)
df.to_sql('Crime_Type', con, if_exists='replace', index=False)

#add sub tables
con = sqlite3.connect('DC22_DB.db')
query = f'''SELECT DISTINCT(Year)
                FROM Street_fixed
             '''
sql_query = pd.read_sql_query(query, con)
df = pd.DataFrame(sql_query)
df.to_sql('Years', con, if_exists='replace', index=False)



C:\Users\20192830\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
